In [3]:
import requests
import pandas as pd
from configparser import ConfigParser

In [4]:
# Credentials

# Email: 'ds4002project1@gmail.com'
# Email password: 'DS4002Project1'

# Reddit username: ds4002project1
# Reddit password: DataScienceProject1

In [5]:
# Public credentials
username = 'ds4002project1'
client_id = 'Zv7btbANVbo3H8hOoSjFOw'

# Private credentials (will need to secure eventually)
pw = 'DataScienceProject1'
secret_key = 'PcKxP3QTk-Bc6wCBYXFWXEqsubrAjw'

In [6]:
# Ensuring our credentials are valid
auth = requests.auth.HTTPBasicAuth(client_id, secret_key)

In [7]:
data = {
    'grant_type': 'password',
    'username': username,
    'password': pw
}

headers = {'User-Agent': 'My DS4002 API'}

In [8]:
res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
token = res.json()['access_token']

In [9]:
# Authorizing with token
headers['Authorization'] = f'bearer {token}'

In [8]:
# r/politics, r/news, r/worldnews, r/conservative, r/liberal
sub = input('What subreddit would you like to fetch posts from?: ')
limit = int(input('How many posts would you like to retrieve (1-100)?: '))
time = input('I would like to view the top posts from the past (hour, day, week, month, year, all): ').lower()

What subreddit would you like to fetch posts from?:  liberal
How many posts would you like to retrieve (1-100)?:  100
I would like to view the top posts from the past (hour, day, week, month, year, all):  year


In [9]:
# Requesting data as a json
try:
    r = requests.get(f'https://oauth.reddit.com/r/{sub}/top', headers=headers, params={'t': time, 'limit': limit}).json()
except:
    print('Unable to fetch posts')

In [10]:
post_list = r['data']['children']
len(post_list) # post_list is a list with each element corresponding to a post

100

In [11]:
# Creating a blank dataframe (pandas library) which we can populate with our data
df = pd.DataFrame(data={'Title': [], 'ID': [], 'Number of Upvotes': [], 'Percent Upvoted': [], 'Number of Comments': [], 
                        'Link': []}).astype({"Number of Upvotes": int, "Number of Comments": int})

In [12]:
for i in range(0, len(post_list)): # Iterate through the number of posts specified by user
    data = post_list[i]['data']
    # For each post grab the relevant data
    if 'url_overridden_by_dest' in data.keys(): # If there is a url to an article provided we want it
        temp_df = pd.DataFrame(data={'Title': data['title'], 'ID': data['name'], 'Number of Upvotes': int(data['score']), 
                                     'Percent Upvoted': data['upvote_ratio'], 
                                     'Number of Comments': int(data['num_comments']), 
                                     'Link': data['url_overridden_by_dest']}, index=[0])
    else: # If no url
        temp_df = pd.DataFrame(data={'Title': data['title'], 'ID': data['name'], 'Number of Upvotes': int(data['score']), 
                                     'Percent Upvoted': data['upvote_ratio'], 
                                     'Number of Comments': int(data['num_comments']), 
                                     'Link': 'None'}, index=[0])
    df = df.append(temp_df) # This is kinda janky there's probably a better way to do this
df = df.reset_index(drop=True)
df.head(5) # Here's what the beginning of our dataframe looks like

,Title,ID,Number of Upvotes,Percent Upvoted,Number of Comments,Link
0,Pro-Trump Election Deniers Lose in Nebraska Re...,t3_undhn8,1083,0.95,76,https://www.bloomberg.com/news/articles/2022-0...
1,Most Americans support using the popular vote ...,t3_wku42m,970,0.95,152,https://www.npr.org/2022/08/10/1116688726/most...
2,"The Republican Party Needs To Be Abolished, No...",t3_uyril3,881,0.90,221,https://www.msnbc.com/msnbc/amp/rcna30497
3,Dems Should Sue Now To Stop Trump Run Under Co...,t3_w21mou,772,0.95,85,https://www.huffpost.com/entry/14th-amendment-...
4,Bill Barr Reportedly Burns His Bridges in Scat...,t3_t2q9eo,725,0.98,92,https://www.mediaite.com/trump/bill-barr-repor...


In [13]:
# Messing around with potential quantitative metrics
df['Upvote to Comment Ratio'] = round(df['Number of Upvotes'] / df['Number of Comments'], 2)
df['Controversy Rating'] = 1 / (df['Percent Upvoted'] * df['Upvote to Comment Ratio'])
df

,Title,ID,Number of Upvotes,Percent Upvoted,Number of Comments,Link,Upvote to Comment Ratio,Controversy Rating
0,Pro-Trump Election Deniers Lose in Nebraska Re...,t3_undhn8,1083,0.95,76,https://www.bloomberg.com/news/articles/2022-0...,14.25,0.073869
1,Most Americans support using the popular vote ...,t3_wku42m,970,0.95,152,https://www.npr.org/2022/08/10/1116688726/most...,6.38,0.164989
2,"The Republican Party Needs To Be Abolished, No...",t3_uyril3,881,0.90,221,https://www.msnbc.com/msnbc/amp/rcna30497,3.99,0.278474
3,Dems Should Sue Now To Stop Trump Run Under Co...,t3_w21mou,772,0.95,85,https://www.huffpost.com/entry/14th-amendment-...,9.08,0.115929
4,Bill Barr Reportedly Burns His Bridges in Scat...,t3_t2q9eo,725,0.98,92,https://www.mediaite.com/trump/bill-barr-repor...,7.88,0.129493
...,...,...,...,...,...,...,...,...
95,Many Believe It's Time To Do Away With Lawmake...,t3_pv3rt5,433,1.00,33,https://www.npr.org/2021/09/22/1039565467/many...,13.12,0.076220
96,"Trump, Secret Service agent confirmed main ele...",t3_vng3x5,436,0.96,54,https://theweek.com/donald-trump/1014743/trump...,8.07,0.129079
97,Donald Trump Launched His Social Media Platfor...,t3_qeurbs,432,0.97,70,https://www.giantfreakinrobot.com/tech/donald-...,6.17,0.167087
98,POLITICS - NEWS ANALYSIS WaPo Bombshell: Trump...,t3_x05tbc,430,0.97,77,None,5.58,0.184754


In [14]:
# The rest is trying to figure out the commenting

In [15]:
# Each "thing" (post, subreddit, user, etc.) has a unique ID. The number after the t indicates what type of thing
# We need to get the post ID to search through its comments. So for example here's the ID of the first on in the df
post_list[0]['data']['name']

't3_undhn8'

In [16]:
post_list[0]['data']['title']

'Pro-Trump Election Deniers Lose in Nebraska Republican Primaries'

In [17]:
# One post from the politcs subreddit had the id 't3_x8y56x'. So we can pass the stuff after the underscore into
# the URL like this
r = requests.get(f'https://oauth.reddit.com/r/politics/comments/x8y56x', headers=headers, params={'sort': 'top'}).json()

In [18]:
#df.to_csv('liberal100year.csv', index=False)

In [19]:
# PROCESSING COMMENTS

In [10]:
def get_comments(df, filename, sub_name, sub_has_mod, num_posts=100, num_comments=10):
    comment_list = []
    for i in range(0, num_posts):
        post_id = df.iloc[i]['ID'][3:]
        r = requests.get(f'https://oauth.reddit.com/r/{sub_name}/comments/{post_id}', headers=headers, params={'sort': 'top'}).json()
        for j in range(int(sub_has_mod), num_comments + int(sub_has_mod)): # Exclude automated mod comment if present
            comment_text = r[1]['data']['children'][j]['data']['body']
            comment_text = comment_text.replace('\n', '')
            comment_list.append(comment_text)
    print(len(comment_list))
    with open(filename, 'w', encoding="utf-8") as f:
        for comment in comment_list:
            f.write(comment)
            f.write('\n')
    f.close()
    return True

In [11]:
# politics_df = pd.read_csv('politics100year.csv')
# news_df = pd.read_csv('news100year.csv')
# worldnews_df = pd.read_csv('worldnews100year.csv')
# conservative_df = pd.read_csv('conservative100year.csv')
# liberal_df = pd.read_csv('liberal100year.csv')

In [14]:
# get_comments(liberal_df, 'liberal1010.txt', 'liberal', False, 10, 10)

100


True

In [1]:
# SENTIMENT ANALYSIS

In [11]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [14]:
def sentiment_vader(sentence):
    # https://towardsdatascience.com/the-most-favorable-pre-trained-sentiment-classifiers-in-python-9107c06442c6

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    sentiment_dict = sid_obj.polarity_scores(sentence)
    negative = sentiment_dict['neg']
    neutral = sentiment_dict['neu']
    positive = sentiment_dict['pos']
    compound = sentiment_dict['compound']

    if sentiment_dict['compound'] >= 0.05 :
        overall_sentiment = "Positive"

    elif sentiment_dict['compound'] <= - 0.05 :
        overall_sentiment = "Negative"

    else :
        overall_sentiment = "Neutral"
  
    return negative, neutral, positive, compound, overall_sentiment

In [17]:
sentiment_vader('This is a great, happy sentence')

(0.0, 0.248, 0.752, 0.8442, 'Positive')